In [1]:
## Calculate summary statistics for correlation of gene-pairs

# Libraries

In [2]:
import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
from matplotlib import pyplot as plt
from scipy.sparse import issparse

In [3]:
import os

In [4]:
import scipy.stats as stats
import numpy as np

# Parameters

In [5]:
### Define dataset and cell-type for which to prepare

In [6]:
dataset = 'oneK1k'
dataset2 = 'oneK1K'

In [7]:
cell_type = 'CD8_T'

In [ ]:
### Define chromosome for which to calculate (calculate seperately per chromosome and aggragate then)

In [8]:
chromosome = 'chr-2'

In [9]:
### Path to the correlation files

In [10]:
data_path = '../data/current/coeqtl_mapping/co_qtls_sceqtlgen/correlations/'


## co_qtls_sceqtlgen can be replaced by co_qtls_decision_tree
## co_qtls_sceqtlgen: contains all for initial mapping run
## co_qtls_decision_tree: files for final decision tree filtered run

In [ ]:
### Path to the calculated expression summary stats

In [11]:
data_path_summary_stats = '/lustre/groups/epigenereg01/workspace/projects/grn_dev_groningen/summary_stats/'

## Files: Sample_gene_statistic  (info about genes on sample level)
#         Sample_summary (information about samples)
#         Sample_cell statistic

In [ ]:
### Path to save calculated summaries to

In [12]:
result_path = '../results/current/F6/'

In [13]:
result_path_analysis = "../data/current/coeqtl_mapping/co_qtls_sceqtlgen/analysis_"  + dataset2 + "/" + cell_type + "/"

# Functions

In [14]:
def get_r_thres(n, p_thres=0.975):
    t = stats.t.ppf(p_thres, n)
    return np.sqrt(1 / (((n - 2) / t ** 2) + 1))

In [15]:
## weighted standard deviation

def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- NumPy ndarrays with the same shape.
    """
    weights = weights[np.isnan(values) == False] # remove the nan correlations
    values = values[np.isnan(values) == False]   # remove the nan correlations
    
    average = np.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    #return (math.sqrt(variance)) #to return standard deviation
    return(variance)

# Load Data

## Summary stats: cells per sample

In [ ]:
## Information about number of cells per sample

In [16]:
available_files = os.listdir(data_path_summary_stats)

In [17]:
available_files = pd.Series(available_files)

In [18]:
available_files

0      D1_Sample_cell_statisticwg3_sawcerILC.Qced.Nor...
1      D1_Sample_summarywg3_Franke_split_v3Mono.Qced....
2      D1_Sample_summarywg3_Franke_split_v2CD8_T.nc20...
3      D1_Sample_summarywg3_NawijnEryth.Qced.Normaliz...
4      D1_Sample_gene_statisticwg3_Franke_split_v2HSP...
                             ...                        
529    D1_Sample_cell_statisticwg3_YeNK.Qced.Normaliz...
530    D1_Sample_gene_statisticwg3_Franke_split_v2NK....
531    D1_Sample_summarywg3_Franke_split_v3HSPC.Qced....
532    D1_Sample_summarywg3_NawijnCD4_T.Qced.Normaliz...
533    D1_Sample_cell_statisticwg3_NawijnCD4_T.Qced.N...
Length: 534, dtype: object

In [19]:
available_files = available_files[available_files.str.extract( fr'({dataset})', expand=True).notna()[0]]  # onyl oneK1K info

In [20]:
available_files

25     D1_Sample_cell_statisticwg3_oneK1kCD4_T.Qced.N...
39     D1_Sample_cell_statisticwg3_oneK1kPlatelet.Qce...
47     D1_Sample_cell_statisticwg3_oneK1kCD4_T.Qced.N...
52     D1_Sample_cell_statisticwg3_oneK1kB.Qced.Norma...
63     D1_Sample_gene_statisticwg3_oneK1kCD4_T.Qced.N...
70     D1_Sample_gene_statisticwg3_oneK1kEryth.Qced.N...
74     D1_Sample_gene_statisticwg3_oneK1kCD4_T.Qced.N...
77     D1_Sample_summarywg3_oneK1kEryth.Qced.Normaliz...
81     D1_Sample_gene_statisticwg3_oneK1kMono.Qced.No...
110    D1_Sample_cell_statisticwg3_oneK1kCD8_T.Qced.N...
111    D1_Sample_summarywg3_oneK1kCD4_T.Qced.Normaliz...
115    D1_Sample_cell_statisticwg3_oneK1kB.Qced.Norma...
131    D1_Sample_gene_statisticwg3_oneK1kDC.Qced.Norm...
136    D1_Sample_cell_statisticwg3_oneK1kother_T.Qced...
172    D1_Sample_gene_statisticwg3_oneK1kHSPC.Qced.No...
173    D1_Sample_summarywg3_oneK1kHSPC.Qced.Normalize...
176    D1_Sample_cell_statisticwg3_oneK1kCD4_T.Qced.N...
188    D1_Sample_cell_statistic

In [21]:
available_files = available_files[available_files.str.extract( r'(Sample_summary)', expand=True).notna()[0]]  # onyl sample summaries

In [22]:
available_files = available_files[available_files.str.extract( r'(SCs.Rds)', expand=True).notna()[0]]  # onyl sample summaries

In [23]:
available_files = available_files[available_files.str.extract(  fr'({cell_type})', expand=True).notna()[0]]  # onyl CD4_T

In [24]:
available_files

201    D1_Sample_summarywg3_oneK1kCD8_T.Qced.Normaliz...
330    D1_Sample_summarywg3_oneK1kCD8_T.Qced.Normaliz...
dtype: object

In [25]:
summary_stats_sample = []

In [26]:
for i in available_files:
    data = pd.read_csv(data_path_summary_stats + i)
    data['source'] = i
    
    summary_stats_sample.append(data)

    

In [27]:
summary_stats_sample = pd.concat(summary_stats_sample)  # contains the amount of cells per sample

In [28]:
summary_stats_sample

,Unnamed: 0,Assignment,amount_cells,dataset,cell_type,source
0,1,1,198,wg3_oneK1k,CD8_T.Qced.Normalized.SCs.Rds,D1_Sample_summarywg3_oneK1kCD8_T.Qced.Normaliz...
1,2,10,60,wg3_oneK1k,CD8_T.Qced.Normalized.SCs.Rds,D1_Sample_summarywg3_oneK1kCD8_T.Qced.Normaliz...
2,3,1000,159,wg3_oneK1k,CD8_T.Qced.Normalized.SCs.Rds,D1_Sample_summarywg3_oneK1kCD8_T.Qced.Normaliz...
3,4,1001,89,wg3_oneK1k,CD8_T.Qced.Normalized.SCs.Rds,D1_Sample_summarywg3_oneK1kCD8_T.Qced.Normaliz...
4,5,1002,151,wg3_oneK1k,CD8_T.Qced.Normalized.SCs.Rds,D1_Sample_summarywg3_oneK1kCD8_T.Qced.Normaliz...
...,...,...,...,...,...,...
1013,1014,995,282,wg3_oneK1k,CD8_T.Qced.Normalized.SCs.Rds,D1_Sample_summarywg3_oneK1kCD8_T.Qced.Normaliz...
1014,1015,996,239,wg3_oneK1k,CD8_T.Qced.Normalized.SCs.Rds,D1_Sample_summarywg3_oneK1kCD8_T.Qced.Normaliz...
1015,1016,997,52,wg3_oneK1k,CD8_T.Qced.Normalized.SCs.Rds,D1_Sample_summarywg3_oneK1kCD8_T.Qced.Normaliz...
1016,1017,998,95,wg3_oneK1k,CD8_T.Qced.Normalized.SCs.Rds,D1_Sample_summarywg3_oneK1kCD8_T.Qced.Normaliz...


In [29]:
summary_stats_sample = summary_stats_sample[['Assignment', 'amount_cells']]

In [30]:
summary_stats_sample

,Assignment,amount_cells
0,1,198
1,10,60
2,1000,159
3,1001,89
4,1002,151
...,...,...
1013,995,282
1014,996,239
1015,997,52
1016,998,95


In [31]:
### Calculate correlation significance threshols

In [32]:
summary_stats_sample['r_thres'] = get_r_thres(summary_stats_sample['amount_cells'])

/tmp/ipykernel_719350/690579974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_stats_sample['r_thres'] = get_r_thres(summary_stats_sample['amount_cells'])


In [33]:
summary_stats_sample

,Assignment,amount_cells,r_thres
0,1,198,0.139481
1,10,60,0.254036
2,1000,159,0.155700
3,1001,89,0.208351
4,1002,151,0.159784
...,...,...,...
1013,995,282,0.116830
1014,996,239,0.126926
1015,997,52,0.273003
1016,998,95,0.201633


In [34]:
summary_stats_sample['Assignment'] =  summary_stats_sample['Assignment'].astype(str)

/tmp/ipykernel_719350/1895368704.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_stats_sample['Assignment'] =  summary_stats_sample['Assignment'].astype(str)


In [35]:
summary_stats_sample

,Assignment,amount_cells,r_thres
0,1,198,0.139481
1,10,60,0.254036
2,1000,159,0.155700
3,1001,89,0.208351
4,1002,151,0.159784
...,...,...,...
1013,995,282,0.116830
1014,996,239,0.126926
1015,997,52,0.273003
1016,998,95,0.201633


## Correlation Data

In [36]:
### Depending on the dataset read in per chromosome

In [38]:
## in case of multiple chromosomes do this per chromosome

In [39]:
if chromosome != '':
     correlation_test =pd.read_csv(data_path + dataset2 + "-" + cell_type + '-' + "pearson-weighted-" + chromosome + "-final.tsv.gz", sep = '\t', compression='gzip')
 #   correlation_test =pd.read_csv(data_path + dataset + "_correlations/" + chromosome + "_1000inds_corr_onek1k_pea_weighted.tsv", sep = '\t')
    
if dataset == 'wijst':
    correlation_test = pd.read_csv(data_path + dataset + "_correlations/" + 'wg3_wijst2018_combined_corr_CD4_T_all_filtered_pearson_weighted.tsv.gz' , sep = '\t')
    
        
if dataset == 'Franke_split_v3':
    correlation_test = pd.read_csv(data_path + dataset + "_correlations/" + 'wg3_Franke_split_v3_combined_corr_CD4_T_top_1000_pearson_weighted.tsv.gz' , sep = '\t')

In [40]:
correlation_test = correlation_test.rename(columns={'-': 'gene_pair'})

In [41]:
correlation_test.shape[0]

6459033

In [ ]:
### For huge files compute in several iterations (splits)

In [42]:
split_var = 500000

In [43]:
splits = correlation_test.shape[0]/ split_var

In [44]:
splits = round(splits + 0.5)

In [45]:
splits

13

In [46]:
summary_stats_correlation = pd.DataFrame()

In [47]:
rows_start = 0

In [48]:
range(1, splits)

range(1, 13)

In [49]:
#seq(1,splits)

In [50]:
for i in range(0, splits): 
    rows_end = min((i+1) * split_var, correlation_test.shape[0])
    
    #if correlation_test.shape[0] > 2500000:
    correlation_test1 = correlation_test[rows_start : rows_end]
    #correlation_test2 = correlation_test[ 2500000:correlation_test.shape[0] ]

    correlation_test_long1 = pd.melt(correlation_test1, id_vars='gene_pair')
    #correlation_test_long2 = pd.melt(correlation_test2, id_vars='gene_pair')

    correlation_test_long1 = correlation_test_long1.rename(columns={'variable': 'Assignment'})
    correlation_test_long1 = correlation_test_long1.rename(columns={'value': 'correlation'})

    #correlation_test_long2 = correlation_test_long2.rename(columns={'variable': 'Assignment'})
    #correlation_test_long2 = correlation_test_long2.rename(columns={'value': 'correlation'})

    correlation_test_long1['abs_correlation'] = abs(correlation_test_long1["correlation"])
    correlation_test_long1['is_na'] = correlation_test_long1['correlation'].isna()
    correlation_test_long1['is_not_na'] = correlation_test_long1['correlation'].notna()

    #correlation_test_long2['abs_correlation'] = abs(correlation_test_long2["correlation"])
    #correlation_test_long2['is_na'] = correlation_test_long2['correlation'].isna()
    #correlation_test_long2['is_not_na'] = correlation_test_long2['correlation'].notna()

    summary_stats_correlation1 = correlation_test_long1.groupby([ 'gene_pair']).agg( mean_correlation =pd.NamedAgg(column="correlation", aggfunc="mean"),
                                                                mean_abs_correlation =pd.NamedAgg(column="abs_correlation", aggfunc="mean"),
                                                                var_correlation =pd.NamedAgg(column="correlation", aggfunc="var"),
                                                                #sd_correlation = pd.NamedAgg(column = "correlation", aggfunc = "stdev"),
                                                                max_correlation =pd.NamedAgg(column="abs_correlation", aggfunc="max"),
                                                                n_NA = pd.NamedAgg(column = 'is_na', aggfunc = 'sum'),
                                                                n_not_NA = pd.NamedAgg(column = 'is_not_na', aggfunc = 'sum'),
                                                                #n_significant = pd.NamedAgg(column = 'is_significant', aggfunc = 'sum'),
                                                                #weighted_variance = pd.NamedAgg(column='correlation', aggfunc=lambda x: weighted_avg_and_std(x, correlation_test_long.loc[x.index, 'amount_cells']))

                                                                                           ).reset_index()

    # (skipna=False)


        #summary_stats_correlation2 = correlation_test_long2.groupby([ 'gene_pair']).agg( mean_correlation =pd.NamedAgg(column="correlation", aggfunc="mean"),
        #                                                            mean_abs_correlation =pd.NamedAgg(column="abs_correlation", aggfunc="mean"),
        #                                                            var_correlation =pd.NamedAgg(column="correlation", aggfunc="var"),
        #                                                            #sd_correlation = pd.NamedAgg(column = "correlation", aggfunc = "stdev"),
        #                                                            max_correlation =pd.NamedAgg(column="abs_correlation", aggfunc="max"),
        #                                                            n_NA = pd.NamedAgg(column = 'is_na', aggfunc = 'sum'),
        #                                                            n_not_NA = pd.NamedAgg(column = 'is_not_na', aggfunc = 'sum'),
        #                                                            #n_significant = pd.NamedAgg(column = 'is_significant', aggfunc = 'sum'),
        #                                                            #weighted_variance = pd.NamedAgg(column='correlation', aggfunc=lambda x: weighted_avg_and_std(x, correlation_test_long.loc[x.index, 'amount_cells']))
#
#                                                                                                  ).reset_index()

    rows_start = 0 + rows_end
    summary_stats_correlation = pd.concat([summary_stats_correlation1, summary_stats_correlation], ignore_index=True)

    # (skipna=False)

In [52]:
summary_stats_correlation

,gene_pair,mean_correlation,mean_abs_correlation,var_correlation,max_correlation,n_NA,n_not_NA
0,RMND5A_SHMT1,0.005981,0.048232,0.004533,0.353163,932,83
1,RMND5A_SHMT2,-0.008068,0.049046,0.003636,0.145758,927,88
2,RMND5A_SHOC2,0.008357,0.059296,0.006084,0.359652,413,602
3,RMND5A_SHPRH,-0.001739,0.056525,0.004940,0.290474,489,526
4,RMND5A_SHQ1,-0.005160,0.048619,0.003612,0.198743,755,260
...,...,...,...,...,...,...,...
6459028,ANKRD44_ENSG00000288880,-0.008393,0.054032,0.004345,0.136053,941,74
6459029,ANKRD44_ENSG00000288891,0.003543,0.059890,0.005984,0.434034,439,576
6459030,ANKRD44_ENSG00000288896,-0.004896,0.052134,0.004310,0.128492,988,27
6459031,ANKRD44_ENSG00000288899,0.001451,0.058759,0.005591,0.273659,627,388


In [56]:
result_path_analysis

'../data/current/coeqtl_mapping/co_qtls_decision_tree/analysis_oneK1K/CD8_T/'

In [57]:
## Save the result

In [58]:
if chromosome != '':
    summary_stats_correlation.to_csv(result_path_analysis +  'F6_Correlation_Summary_Stats_' + chromosome + '.csv')
    
if chromosome == '':
    summary_stats_correlation.to_csv(result_path_analysis +  'F6_Correlation_Summary_Stats_' + 'all.csv')

In [59]:
chromosome

'chr-2'

In [60]:
cell_type

'CD8_T'